In [1]:
import random

In [2]:
class MempoolTransaction():
  def __init__(self, txid, fee, weight, parents):
    self.txid = txid
    self.fee = int(fee)
    self.weight = int(weight)
    self.parents = parents 

In [3]:
def parse_mempool_csv():
  """Parse the CSV file and return a list of MempoolTransactions."""
  with open('mempool.csv') as f:
    return([MempoolTransaction(*line.strip().split(',')) for line in f.readlines()[1:]])

In [4]:
global MAXIMUM_BLOCK_WEIGHT 
MAXIMUM_BLOCK_WEIGHT = 4000000

In [5]:
def create_block_file(blockTxns):
  """Creates the block.txt file"""
  textfile = open("/content/block.txt", "w")
  for element in blockTxns:
    textfile.write(element + "\n")
  textfile. close()

We define TXN density as:

$\text{TXN density} = \frac{\text{TXN fee}}{\text{TXN size}}$

where
$\text{TXN size} = \frac{\text{TXN weight}}{\text{BLOCK_WEIGHT}}$

Here, BLOCK_WEIGHT = 4000000

In [6]:
def compute_helper_mempool_dictionaries():
  """Computes helper dictionary and TXN density"""
  txn_density_dict = {}
  txn_parents_dict = {}
  txn_size_dict = {}
  mempool_data = parse_mempool_csv()
  for elem in mempool_data:
    size = elem.weight/MAXIMUM_BLOCK_WEIGHT # weight mapped to (0,1)
    txn_size_dict[elem.txid] = size 
    txn_density_dict[elem.txid] = elem.fee/size
    if elem.parents != '':
      txn_parents_dict[elem.txid] = elem.parents.strip().split(';')
  return txn_density_dict,txn_parents_dict,txn_size_dict

In [7]:
def order_TXN_density(txn_density_dict):
  """Order Transactions based on descending TXN density"""
  sorted_txn_density = sorted(txn_density_dict.items(), key=lambda x: x[1], reverse=True)
  return sorted_txn_density

In [8]:
def create_valid_block(sorted_txn_density,txn_parents_dict,txn_size_dict):
  """This is where the magic happens"""
  currentBlockSize = 0
  blockTxns = []
  while(len(sorted_txn_density) != 0):
    currentTxn = sorted_txn_density[0]

    if currentTxn[0] in blockTxns:
      sorted_txn_density.pop(0)
      continue

    if currentTxn[0] in txn_parents_dict:
      parents = txn_parents_dict[currentTxn[0]]
      rand = random.random()
      if rand <= 0.5:
        totalParentsTxnSize = 0
        for parent in parents:
          if parent not in blockTxns:
            totalParentsTxnSize = totalParentsTxnSize + txn_size_dict[parent]
        if currentBlockSize + totalParentsTxnSize > 1:
          sorted_txn_density.pop(0)
          continue
        else:
          currentBlockSize = currentBlockSize + totalParentsTxnSize
          for parent in parents:
            if parent not in blockTxns:
              blockTxns.append(parent)
            blockTxns.append(currentTxn[0])
            sorted_txn_density.pop(0)
          continue
      else:
        sorted_txn_density.pop(0)
        continue

    if currentBlockSize + txn_size_dict[currentTxn[0]] >= 1:
      break
    else:
      currentBlockSize = currentBlockSize + txn_size_dict[currentTxn[0]]
    blockTxns.append(currentTxn[0])
    sorted_txn_density.pop(0)
  return currentBlockSize,blockTxns

In [9]:
def create_valid_block_recur(sorted_txn_density,currentBlockSize,txn_density_dict,blockTxns,currentTxn,reward,maxReward,bestBlockTxns,*args):
  if currentTxn in blockTxns:
    sorted_txn_density.pop(0)
    return currentBlockSize, create_valid_block_recur(sorted_txn_density,currentBlockSize,txn_density_dict,blockTxns,sorted_txn_density(0),reward,maxReward,bestBlockTxns,'NONE')

  if len(sorted_txn_density) == 0:
    return currentBlockSize, blockTxns

  newReward = 0
  if args[0] == 'PARENT_INCLUDED':
    totalParentsTxnSize = 0
    for parent in txn_parents_dict[currentTxn[0]]:
      if parent not in blockTxns:
        totalParentsTxnSize = totalParentsTxnSize + txn_size_dict[parent]
    if currentBlockSize + totalParentsTxnSize > 1:
        sorted_txn_density.pop(0)
        return currentBlockSize, blockTxns
    else:
      currentBlockSize = currentBlockSize + totalParentsTxnSize
      for parent in txn_parents_dict[currentTxn[0]]:
        if parent not in blockTxns:
          newReward = reward + txn_density_dict[parent]
          blockTxns.append(parent)
        blockTxns.append(currentTxn[0])

  if args[0] == 'PARENT_NOT_INCLUDED':
    if currentBlockSize + txn_size_dict[currentTxn[0]] > 1:
      sorted_txn_density.pop(0)
      return currentBlockSize, blockTxns
    else:
      currentBlockSize = currentBlockSize + txn_size_dict[currentTxn[0]]
      newReward = reward + currentTxn[1]
      blockTxns.append(currentTxn[0])

  if newReward > maxReward:
    maxReward = newReward
    bestBlockTxns = blockTxns

  sorted_txn_density.pop(0)
  currentTxn = sorted_txn_density[0]

  if currentTxn[0] in txn_parents_dict:
    return currentBlockSize, create_valid_block_recur(sorted_txn_density,currentBlockSize,txn_density_dict,blockTxns,currentTxn,newReward,maxReward,bestBlockTxns,'PARENT_INCLUDED')
  else:
    return currentBlockSize, create_valid_block_recur(sorted_txn_density,currentBlockSize,txn_density_dict,blockTxns,currentTxn,newReward,maxReward,bestBlockTxns,'PARENT_NOT_INCLUDED')

In [10]:
if __name__ == "__main__":
  txn_density_dict,txn_parents_dict,txn_size_dict = compute_helper_mempool_dictionaries()
  sorted_txn_density = order_TXN_density(txn_density_dict)
  currentBlockSize,blockTxns = create_valid_block(sorted_txn_density,txn_parents_dict,txn_size_dict)
  print('Current Block Size is ',currentBlockSize*MAXIMUM_BLOCK_WEIGHT)
  create_block_file(blockTxns)

Current Block Size is  3999963.99999996


In [11]:
# #Recursive
# if __name__ == "__main__":
#   txn_density_dict,txn_parents_dict,txn_size_dict = compute_helper_mempool_dictionaries()
#   sorted_txn_density = order_TXN_density(txn_density_dict)
#   maxReward = 0
#   bestBlockTxns = []
#   currentBlockSize = 0
#   blockTxns = []
#   currentTxn = sorted_txn_density[0]
#   if currentTxn[0] in txn_parents_dict:
#     currentBlockSize,blockTxns = create_valid_block_recur(sorted_txn_density,currentBlockSize,txn_density_dict,blockTxns,currentTxn,0,maxReward,bestBlockTxns,'PARENT_INCLUDED')
#   else:
#     currentBlockSize,blockTxns = create_valid_block_recur(sorted_txn_density,currentBlockSize,txn_density_dict,blockTxns,currentTxn,0,maxReward,bestBlockTxns,'PARENT_NOT_INCLUDED')